In [1]:
import numpy as np
import os
import glob
from global_land_mask import globe

def process_pkl_file(file_path):
    data = np.load(file_path, allow_pickle=True)
    lon = data[:, 0, ]  
    lat = data[:, 1, ]  
    height = data[:, 2, ]  
    bl_height = data[:, 3, ]  
    time_steps = data.shape[2] 
    # Normalize longitude to the range [-180, 180]
    # Not sure why this happend but this seems to fix it
    lon = np.where(lon > 180, lon - 360, lon)
    return lon, lat, height, bl_height, time_steps

def filter_points(lon, lat, height, bl_height):
    # Filter land points
    is_on_land = globe.is_land(lat, lon)
    
    lon_land = lon[is_on_land]
    lat_land = lat[is_on_land]
    height_land = height[is_on_land]
    bl_height_land = bl_height[is_on_land]
    
    return lon_land, lat_land, height_land, bl_height_land

def filter_by_bl_height(lon, lat, height, bl_height):
    below_bl = []
    above_bl = []
    for lon_point, lat_point, h, bl_h in zip(lon, lat, height, bl_height):
        if h <= bl_h:
            below_bl.append((lat_point, lon_point))
        else:
            above_bl.append((lat_point, lon_point))
    return below_bl, above_bl

def analyze_pkl_file(pkl_file):
    if os.path.exists(pkl_file):
        lon, lat, height, bl_height, time_steps = process_pkl_file(pkl_file)
        

        lon_land, lat_land, height_land, bl_height_land = filter_points(lon.ravel(), lat.ravel(), height.ravel(), bl_height.ravel())

        below_bl, above_bl = filter_by_bl_height(lon_land, lat_land, height_land, bl_height_land)
        return below_bl, above_bl, time_steps
    else:
        print(f"File not found: {pkl_file}")
        return [], [], 0

def process_all_pkl_files(directory, output_file):
    pkl_files = glob.glob(os.path.join(directory, '*_part_poss.pkl'))
   
    with open(output_file, 'w') as file:
        for pkl_file in pkl_files:
            all_below_bl, all_above_bl, time_steps = analyze_pkl_file(pkl_file)
            
            total_points = len(all_below_bl) + len(all_above_bl)
            percent_below_bl = (len(all_below_bl) / total_points * 100) if total_points > 0 else 0
            percent_above_bl = (len(all_above_bl) / total_points * 100) if total_points > 0 else 0

            file.write(f"'{pkl_file}', : , {percent_below_bl:.2f}\n")
            # file.write(f"Results for file: {os.path.basename(pkl_file)}\n")
            # file.write(f"Points below BL on land: {len(all_below_bl)}\n")
            # file.write(f"Points above BL on land: {len(all_above_bl)}\n")
            # file.write(f"Total points on land: {total_points}\n")
            # file.write(f"Total time steps analyzed: {time_steps}\n")
            # file.write(f"Percentage of points below BL on land: {percent_below_bl:.2f}%\n")
            # file.write(f"Percentage of points above BL on land: {percent_above_bl:.2f}%\n")
            # file.write("\n")

if __name__ == "__main__":
    directory = 'pkl_files/MPHASE/AIRTRACER'
    output_file = 'boundary_layer_analysis_results_AREO.txt'
    process_all_pkl_files(directory, output_file)